In [9]:
import re, subprocess, pyodbc 
import numpy as np # linear algebra 
from sqlalchemy import func, create_engine
from config import mongoConn, connstr  
import pandas as pd  
from bs4 import BeautifulSoup


In [10]:
engine = create_engine(connstr) 
def get_df(query):
    conn = engine.connect() 
    df = pd.read_sql(query,con=conn) 
    conn.close()
    return df

In [2]:
def id_num(s):
    if s == '':
        return 0
    s = s.strip().lower().replace(' ', '')
    l = ['a','b','c','d','e','f','g','h']
    d = [.100, 1, 100, 10000]
    v = [0,0,0,0]
    for i, c in enumerate(l):
        s = s.replace(c,f'{i}') 
    l = s.split('.')
    for i, c in enumerate(l):
        v[i] =  int(c) / d[i] 
    return sum(v)  
print(id_num('1.1.1')) 
print(id_num('1.1.10.1')) 

11.01
11.1001


In [3]:
def normalizedf(df):
    df.drop( columns=[c for c in df.columns if '__' in c ] , inplace=True )   
    cols = list(df)   
    df[cols] = df[cols].astype(str)
    df['IDNUM'] = df['IDTEXT'].apply(id_num)
    df.sort_values('IDNUM',inplace=True)    
    return df

In [78]:
query = f"""  
SELECT PK_Question, IdText as IDTEXT FROM vwQuestions
WHERE QGROUP=3270
    """  
df = get_df(query) 

In [111]:
query = f"""  
SELECT CONVERT(NVARCHAR(4),PK_PickList)+'',  LEFT(DisplayValue, 15) DisplayValue  FROM vwPickLists
WHERE PK_PickListType=153
    """  
dfpkt = get_df(query) 
# dfpkt.to_clipboard(sep=',') 

In [98]:
query = f"""  
SELECT CONVERT(NVARCHAR(4),PK_PickList)+'',  LEFT(DisplayValue, 15) DisplayValue  FROM vwPickLists
WHERE PK_PickListType=356
    """  
dfpk = get_df(query) 
dfpk[2:4]

,,DisplayValue
2,4252,Agency support
3,4253,Business Intell


In [80]:
def re_id(s):
    s=s.replace('_','.').replace('r-m-','')  
    r=df.loc[df.IDTEXT==s, 'PK_Question'] 
    if len(r)==1:
        return r.item() 
    return s
re_id('1.1.1')

28290

In [104]:
l=[]
s=''
with open(fr'D:\dev\CyberScope\CyberScopeBranch\CSwebdev\code\CyberScope\HVA\2022\2022_A_HVA_1A.aspx', mode="r") as f:
    s = f.read()    
soup = BeautifulSoup(s, 'html.parser') 
trs = soup.select("tr[data-question_master*=r-m-]")
for e in trs[:]: 
    l.append({'i':e['id'],'m':e['data-question_master'],'v':e['data-value_torequire']}) 
dff = pd.DataFrame(l)
dff['m']=dff['m'].apply(re_id)
dff['idt']=dff['i'].apply(lambda s: s.replace('_','.').replace('r-m-','') )
dff['i']=dff['i'].apply(re_id)

dff

,i,m,v,idt
0,28290,28202,.GOV Domain Reg,1.1.1
1,28291,28202,Access to Data,1.1.2
2,28292,28202,Agency support,1.1.3
3,28293,28202,Business Intell,1.1.4
4,28294,28202,Collaboration T,1.1.5
...,...,...,...,...
87,28278,28276,^Yes$|^Y$,7.6.2.1
88,28279,28255,Client,0.7.7
89,28280,28255,Client Server,7.7.1
90,28282,28280,Other,7.8.1


In [102]:
dff

,i,m,v
0,28290,28202,.GOV Domain Reg
1,28291,28202,Access to Data
2,28292,28202,Agency support
3,28293,28202,Business Intell
4,28294,28202,Collaboration T
...,...,...,...
87,28278,28276,^Yes$|^Y$
88,28279,28255,Client
89,28280,28255,Client Server
90,28282,28280,Other


In [107]:
INS = f"""INSERT INTO [fsma_QuestionDependencies] 
(FK_Question, FK_Question_Master, Question_Master_CodeValue_ToMakeRequired) VALUES 
    """  
sql = ""    
for i, r in dff[:].iterrows():  
    v=r['v'].replace('^Yes$|^Y$','Y')
    v=v.replace('$','').replace('^','')
    if i <=50:
        v=i+4250
    s = f"({r['i']}, {r['m']}, '{v}') , -- {r['idt']}, {r['v']}"     
    sql = sql + s + '\n' 
sql = f'{INS}\n {sql}\n' 
print(sql)

INSERT INTO [fsma_QuestionDependencies] 
(FK_Question, FK_Question_Master, Question_Master_CodeValue_ToMakeRequired) VALUES 
    
 (28290, 28202, '4250') , -- 1.1.1, .GOV Domain Reg
(28291, 28202, '4251') , -- 1.1.2, Access to Data
(28292, 28202, '4252') , -- 1.1.3, Agency support
(28293, 28202, '4253') , -- 1.1.4, Business Intell
(28294, 28202, '4254') , -- 1.1.5, Collaboration T
(28295, 28202, '4255') , -- 1.1.6, Collect, analyz
(28296, 28202, '4256') , -- 1.1.7, Consulate Datab
(28297, 28202, '4257') , -- 1.1.8, Cybersecurity M
(28298, 28202, '4258') , -- 1.1.9, Cybersecurity t
(28299, 28202, '4259') , -- 1.1.10, Data Repository
(28300, 28202, '4260') , -- 1.1.11, Decision making
(28301, 28202, '4261') , -- 1.1.12, Diplomatic and 
(28302, 28202, '4262') , -- 1.1.13, Email services 
(28303, 28202, '4263') , -- 1.1.14, Emergency manag
(28304, 28202, '4264') , -- 1.1.15, Emergency respo
(28305, 28202, '4265') , -- 1.1.16, Enterprise reso
(28306, 28202, '4266') , -- 1.1.17, Explosive Th

In [62]:
sql = "" 
for i, r in dff[:].iterrows(): 
    s = "(@QID, @FormName, @PK_QGroup, @SORD, N'@IDTEXT', @QTYPE, 1, @PKTYPE, 0, NULL, N'@QTEXT','@HTEXT'),"
    s = s.replace('@QID',  str(r['QPK']))
    s = s.replace('@SORD', str(i+1)) 
    s = s.replace('@QTYPE', r['FK_QuestionType']) 
    s = s.replace('@PKTYPE', r['FK_PickListType']) 
    s = s.replace('@IDTEXT', r['IDTEXT']) 
    s = s.replace('@QTEXT', r['QuestionText'].replace('\'', '`').replace('\ufffd', ''))  
    s = s.replace('@HTEXT', r['help_text'])  
    sql = sql + s + '\r'
sql = sql.replace("'NULL')", "NULL)")    
sql = sql.replace("N'NULL'", "N''")   
sql = sql.replace("None,", "NULL,")  
sql = f'{INS}\n {sql}\n' 
with open(fr'C:\Users\Tim\Documents\1802\HVA\update.sql', 'w') as f:
    f.write(sql)  



df = pd.read_excel(r'C:\Users\Tim\Documents\1802\HVA\HVA1.xlsx', sheet_name='PurposePickList').fillna('NULL') 
cols = list(df) # Creates list of all column headers
df[cols] = df[cols].astype(str)
df['DisplayCode'] =  df['DisplayValue'].apply(lambda s: re.sub('[^A-Z0-9]','',s.upper()))
df['DisplayCode'] =  df['DisplayCode'].apply(lambda s: re.sub('[AEIOURST]','',s)[:8])
s =''
for i,r, in df.iterrows():
    PK_PickList=r['PK_PickList']
    DisplayCode=r['DisplayCode']
    DisplayValue=r['DisplayValue'] 
    if DisplayValue=='Other':
        DisplayCode='OTH'
    s = s + f"({PK_PickList},@PKT, N'{DisplayCode}',N'{DisplayValue}',{i}, GETDATE(), 1),\n"

print(s)


In [77]:
query = f""" \
    SELECT TOP 5000 
      IDTEXT AS IDTEXT 
    , PK_Question 
    FROM vwQuestions WHERE PK_FORM LIKE '2022-A-HVA'  
"""  
dfdep = get_df(query)  
# dfdep.to_csv(r'C:\Users\Tim\Documents\1802\HVA\dfdep.csv', index=False)  
dfdep[:] 

,IDTEXT,PK_Question
0,0.1,28200
1,1,28201
2,1.1,28202
3,2,28228
4,2.1,28229
...,...,...
103,1.1.45,28334
104,1.1.46,28335
105,1.1.47,28336
106,1.1.48,28337
